In [1]:
from pynq import Overlay
import numpy as np
from pynq import MMIO
import os 
from pynq import allocate
import math
import pynq.lib.dma
import time
from pynq import PL
import asyncio
from pynq import Interrupt

# 导入硬件工程

In [2]:
overlay = Overlay("/home/xilinx/pynq/overlays/base/Accelerator/Accelerator_bigger.bit")
CNN_Control=overlay.CNN_ACCELERATOR.CNN_Control_1 
Windows_Convert=overlay.CNN_ACCELERATOR.Windows_Data_Convert_0
dma_result=overlay.axi_dma_result       
dma_picture=overlay.axi_dma_picture_window 
dma_weight=overlay.axi_dma_weight      
dma_weight_fc=overlay.axi_dma_weight_full 

CONV_DATA_SEND = dma_picture.sendchannel
CONV_WEIGHT_SEND = dma_weight.sendchannel

POOL_DATA_SEND = dma_picture.sendchannel

FC_DATA_SEND = dma_weight.sendchannel
FC_WEIGHT_SEND = dma_weight_fc.sendchannel

RESULT_RECV= dma_result.recvchannel

CNN_Done=Interrupt('CNN_ACCELERATOR/CNN_Control_1/cnn_done')
async def handle():
    await CNN_Done.wait()
    end=time.time()
    cost_time=round(end-start,5)
    print('-------------time cost is--------------- '+ str(cost_time)+'s')
Interrupt_Control_address=0x4120_0000
Interrupt_Control_range=0xFFF
Interrupt_mmio=MMIO(Interrupt_Control_address,Interrupt_Control_range)
Interrupt_Data_address=0x0000
Interrupt_Enable_address=0x0128
Interrupt_GEnable_address=0x011c
Interrupt_state_address=0x0120
Interrupt_mmio.write(Interrupt_Enable_address,0x1)
Interrupt_mmio.write(Interrupt_GEnable_address,0x1)
Interrupt_mmio.write(Interrupt_Enable_address,0x1)

In [3]:
def Get_Interrupt_settings():
#     print("done interrupt state is ",Interrupt_mmio.read(Interrupt_state_address))
    CNN_Control_mmio.write(cnn_start_address  ,1)
    while(Interrupt_mmio.read(Interrupt_state_address)==0): 
        Interrupt_Enable_address=0x0128
    end=time.time()
    cost_time=round(end-start,5)
    Interrupt_mmio.write(Interrupt_state_address,0x1)  #清除标志
    print('-------------time cost is--------------- '+ str(cost_time)+'s')
        

# 地址定义

In [4]:
#-------------------------------------------CNN Control模块-----------------------------------------
CNN_Control_address=0x4000_0000
CNN_Control_range=0xFFF
CNN_Control_mmio=MMIO(CNN_Control_address,CNN_Control_range)

#---------------CNN寄存器地址------------------
Conv_Kernel_Size_address=0x0;
Conv_Kernel_Num_address=0x4;
Pool_Kernel_Size_address=0x8;
Pool_Kernel_Num_address=0xc;
FULLCON_Input_Size_address=0x10;
FULLCON_Output_Size_address=0x14;
select_mode_address=0x18;
cnn_start_address=0x1c;
#relu的缩放这个版本是直接写在硬件里面的，后续量化使用缩放可以使用可以添加，进行动态控制如下：scaler_address=0x20;  scalel_address=0x24;
#---------------CNN mode------------------
MODE_IDLE      =0b00000
MODE_CONV      =0b00001
MODE_RELU      =0b00010
MODE_POOL_Max  =0b00100
MODE_POOL_Mean =0b00101
MODE_FULLCON   =0b01000
MODE_CONV_RELU =0b10000

#---------------寄存器参量------------------
Conv_Kernel_Size   =4;
Conv_Kernel_Num    =4;
Pool_Kernel_Size   =4;
Pool_Kernel_Num    =1;
FULLCON_Input_Size =28;
FULLCON_Output_Size=8;
select_mode        =MODE_CONV;
cnn_start          =0;

#-------------------------------------------滑动窗口-----------------------------------------
Windows_Data_address=0x4000_1000
Windows_Data_range=0xFFF
Windows_mmio=MMIO(Windows_Data_address,Windows_Data_range)
Data_horizontal_address=0x0;
Data_vertical_address=0x4;
Kernel_size_address=0x8;
Stride_address=0xc;
Refresh_address=0x10;

Data_vertical  =28
Data_horizontal=28
Stride          =2
Refresh         =0

In [5]:
def Sliding_window_settings(Data_horizontal,Data_vertical,Kernel_size,Stride=2,Refresh=0):
    Windows_mmio.write(Kernel_size_address,Kernel_size)
    Windows_mmio.write(Data_vertical_address,Data_vertical)
    Windows_mmio.write(Data_horizontal_address,Data_horizontal)
    Windows_mmio.write(Stride_address,Stride)
    Windows_mmio.write(Refresh_address,Refresh)
    
    Kernel_size=Windows_mmio.read(Kernel_size_address)
    Data_vertical=Windows_mmio.read(Data_vertical_address)
    Data_horizontal=Windows_mmio.read(Data_horizontal_address)
    Stride=Windows_mmio.read(Stride_address)
    Refresh=Windows_mmio.read(Refresh_address)
    print("滑窗模块 Kernel_size核心大小      =",Kernel_size)
    print("滑窗模块 Data_vertical图像纵向长  =",Data_vertical)
    print("滑窗模块 Data_horizontal图像横向长=",Data_horizontal)
    print("滑窗模块 Stride步进               =",Stride)
    print("滑窗模块 Refresh刷新              =",Refresh)
  

In [6]:
def Convolutional_settings(mode,Kernel_Size,Kernel_Num,Stride,conv_weight_buffer,Data_horizontal,Data_vertical,input_picture_buff,Weight_refresh=1):
    #滑动窗设置
    
    global Conv_input_data_vertical  
    global Conv_input_data_horizontal
    global Conv_Stride
    global Conv_Kernel_Size
    global Conv_Kernel_Num
    global start
    Conv_input_data_vertical   = Data_vertical  
    Conv_input_data_horizontal = Data_horizontal
    Conv_Stride =Stride
    Conv_Kernel_Size = Kernel_Size
    Conv_Kernel_Num = Kernel_Num

    Sliding_window_settings(Conv_input_data_horizontal,Conv_input_data_vertical,Conv_Kernel_Size,Conv_Stride,0)
    
    #CNN 模块设置
    if(mode==MODE_CONV):
        select_mode=MODE_CONV
    else :
        select_mode=MODE_CONV_RELU
    CNN_Control_mmio.write(select_mode_address        ,select_mode        )
    CNN_Control_mmio.write(Conv_Kernel_Size_address   ,Conv_Kernel_Size   )
    CNN_Control_mmio.write(Conv_Kernel_Num_address    ,Conv_Kernel_Num    )
    print ( " 卷积核心大小    ",   Conv_Kernel_Size    )
    print ( " 卷积核心个数    ",   Conv_Kernel_Num     ) 

    #----------------------------传递权重----------------
    if(Weight_refresh==1):
        CONV_WEIGHT_SEND.transfer(conv_weight_buffer)
        CONV_WEIGHT_SEND.wait()
    #----------------------------传递激励数据----------------
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    

    
    return


In [7]:
def Pooling_settings(mode,Data_horizontal,Data_vertical,input_picture_buffer,Kernel_Size,Kernel_Num=1):
   #滑动窗设置
    global Pool_input_data_vertical   
    global Pool_input_data_horizontal 
    global Pool_Kernel_Size
    global Pool_Kernel_Num 
    global Pool_Stride 
    global start
    Pool_input_data_vertical =Data_vertical  
    Pool_input_data_horizontal =Data_horizontal
    Pool_Kernel_Size =Kernel_Size
    Pool_Kernel_Num  =Kernel_Num
    Pool_Stride      =Kernel_Size
    
    Sliding_window_settings(Pool_input_data_horizontal,Pool_input_data_vertical,Pool_Kernel_Size,Pool_Stride,0)
    if(mode==MODE_POOL_Max):
        select_mode=MODE_POOL_Max
    else :
        select_mode=MODE_POOL_Mean
    CNN_Control_mmio.write(select_mode_address        ,select_mode        )
    CNN_Control_mmio.write(Pool_Kernel_Size_address   ,Pool_Kernel_Size   )
    CNN_Control_mmio.write(Pool_Kernel_Num_address    ,Pool_Kernel_Num    )
    print ( " 卷积核心大小    ",   Pool_Kernel_Size    )
    print ( " 卷积核心个数    ",   Pool_Kernel_Num     ) 

    #----------------------------传递激励数据----------------
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    
    return 

In [8]:
def Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer,FC_act_buffer):
    #滑动窗设置
    global FULLCON_Input_Size   
    global FULLCON_Output_Size  
    global start
    FULLCON_Input_Size    =Input_Size  
    FULLCON_Output_Size   =Output_Size
    #CNN 模块设置
    if(mode==MODE_FULLCON):
        select_mode=MODE_FULLCON
    else :
        select_mode=MODE_FULLCON
    CNN_Control_mmio.write(select_mode_address        ,select_mode        )

    #----------------------------传递权重----------------
    FC_WEIGHT_SEND.transfer(FC_weight_buffer)
    FC_WEIGHT_SEND.wait()
    #----------------------------传递激励数据----------------
    FC_DATA_SEND.transfer(FC_act_buffer)
    FC_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    

    return 

In [9]:
def Show_result_data(select_mode):
    global matrix_conv
    global result_buffer 
    #-----------------------------------------卷积/卷积+激活-------------------------------------    
    if(select_mode==MODE_CONV  or select_mode==MODE_CONV_RELU    ):
        print ( " CNN计算模式  :CONV   ")
        CONV_hang_size=math.floor((Conv_input_data_horizontal-Conv_Kernel_Size)/Conv_Stride+1)
        print("结果输出一行个数---------",CONV_hang_size)
        result_data_size=CONV_hang_size*CONV_hang_size
        print("结果输出总个数---------",result_data_size)
        result_data_size=result_data_size*8#多的空闲位置  16bit*8  
        result_data_size=math.ceil(result_data_size) 
        result_buffer = allocate(shape=(result_data_size,), dtype=np.int16)
        #-------------------------------获取计算结果-------------------------
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait() 
        matrix_conv = np.zeros((8, result_data_size), dtype=np.int16)
        for j in range(int(result_data_size/8)):
            for h in range(8):
                matrix_conv[h][j]=result_buffer[j*8+h]
        
        for h in range(8):
            print("----第", h+1,"个卷积算子结果----","  ", end="")#
            for j in range(int(result_data_size/8)):
                if(h<Conv_Kernel_Num):
                    if(j%CONV_hang_size==0):
                         print("\n")
                    print((matrix_conv[h][j]),"  ", end="")
            print("")    

    #-----------------------------------------池化-------------------------------------               
    if(select_mode==MODE_POOL_Max or select_mode==MODE_POOL_Mean ):
        if(select_mode==MODE_POOL_Max):
            print ( " CNN计算模式   :POOL_Max    ")
        else :
            print ( " CNN计算模式   :POOL_Mean    ")
        Pool_hang_size=math.floor((Pool_input_data_horizontal-Pool_Kernel_Size)/Pool_Kernel_Size+1)
        Pool_data_size=Pool_hang_size*Pool_hang_size
        result_data_size=Pool_data_size*16#多的空闲位置  16bit*8
        matrix_pool = np.zeros((Pool_data_size,), dtype=np.int8)
        result_data_size=math.ceil(result_data_size)
        result_buffer = allocate(shape=(result_data_size,), dtype=np.int8)
        print("结果输出个数---------",result_data_size)
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait()
        for j in range(Pool_data_size):
               matrix_pool[j]=result_buffer[j*16]    
        print("------ 池化算子结果------",  "\n")
        for j in range(Pool_data_size):
            if(j%Pool_hang_size==0):
                 print("\n")
            print(hex( matrix_pool[j]),"  ", end="")
        print("\n")    
#-----------------------------------------全连接-------------------------------------    
    if(select_mode==MODE_FULLCON   ):
        print ( " CNN计算模式   :FULLCON  ")
        result_data_size=8   #多的空闲位置  16bit*8
        result_data_size=math.ceil(result_data_size) 
        result_buffer = allocate(shape=(result_data_size,), dtype=np.int16)
        #-------------------------------获取计算结果-------------------------
        RESULT_RECV.transfer(result_buffer)
        RESULT_RECV.wait() 
        print("------- 全连接算子结果----",  "\n")
        for j in range(FULLCON_Output_Size):
               print(hex( result_buffer[j]),"  ", end="")
        print("\n")


#         return

# 卷积输入激励数据 权重数据

In [10]:
# input_picture_buffer 初始化
Data_vertical=12
Data_horizontal=Data_vertical
Conv_Kernel_Size=5
Stride_Set=1
Conv_Kernel_Num=6
mode_set=MODE_CONV_RELU
Weight_refresh=1

In [11]:
# input_picture_buffer 初始化
Data_vertical=12
Data_horizontal=Data_vertical
Conv_Kernel_Size=5
Stride_Set=1
Conv_Kernel_Num=6
mode_set=MODE_CONV_RELU
Weight_refresh=1
input_picture_data_size = Data_horizontal*Data_vertical
input_picture_buffer = allocate(shape=(input_picture_data_size,), dtype=np.int8)
# input_picture_buffer.reshape((12,12))
for i in range(input_picture_data_size):
    input_picture_buffer[i] = i%Data_vertical+1
data = np.ones((input_picture_data_size,), dtype=np.int8)
# input_picture_buffer=data
print("-------------------------ACT---------------------") 
for j in range(Data_horizontal):
    print("\n")
    for k in range(Data_vertical):
#         if(input_picture_buffer[j*Data_horizontal+k]!=0):
            print(hex(input_picture_buffer[j*Data_horizontal+k]),"  ", end="")
print("\n")           
#input_weight_buffer 初始化
conv_whole_data_size = Conv_Kernel_Size*Conv_Kernel_Size*Conv_Kernel_Num  
input_weight_buffer = allocate(shape=(conv_whole_data_size,), dtype=np.uint8)
conv_weight_buffer = allocate(shape=(Conv_Kernel_Size*Conv_Kernel_Size,), dtype=np.int64)
print("-------------------------WEIGHT---------------------") 
for i in range(Conv_Kernel_Num):
    print ("--------第",Conv_Kernel_Num-i,"个卷积核心------")  
    for j in range(Conv_Kernel_Size):
        for k in range(Conv_Kernel_Size):
            if(i==0):
                input_weight_buffer[i*Conv_Kernel_Size*Conv_Kernel_Size+j*Conv_Kernel_Size+k] =i+1  #(i*Conv_Kernel_Size*Conv_Kernel_Size+j*Conv_Kernel_Size+k)%(Conv_Kernel_Size*Conv_Kernel_Size)+1 ;
            else:
                input_weight_buffer[i*Conv_Kernel_Size*Conv_Kernel_Size+j*Conv_Kernel_Size+k] =i+1 
            print((input_weight_buffer[i*Conv_Kernel_Size*Conv_Kernel_Size+j*Conv_Kernel_Size+k]),"  ", end="") 
        print ("\n")
    print ("\n")    

for j in range(Conv_Kernel_Size*Conv_Kernel_Size ):
    for k in range(Conv_Kernel_Num ):
            conv_weight_buffer[j]=conv_weight_buffer[j]*256+(input_weight_buffer[j+k*Conv_Kernel_Size*Conv_Kernel_Size])
#             print((input_weight_buffer[j*Conv_Kernel_Num+k]),"  ", end="")
    print(hex(conv_weight_buffer[j]),"  ", end="")
    print("-------------------", j,  "\n")

-------------------------ACT---------------------


0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

-------------------------WEIGHT---------------------
-------

In [12]:
print("-------------------------Input Picture---------------------") 
for j in range(Data_horizontal):
    for k in range(Data_vertical):
        if(input_picture_buffer[j*Data_horizontal+k]!=0):
            print(hex(input_picture_buffer[j*Data_horizontal+k]),"  ", end="")
    print("\n")        
for i in range(1):    
    start=time.time()
    Convolutional_settings(mode_set,Conv_Kernel_Size,Conv_Kernel_Num,Stride_Set,conv_weight_buffer,Data_horizontal,Data_vertical,input_picture_buffer,Weight_refresh)
    Show_result_data(mode_set)

-------------------------Input Picture---------------------
0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   

滑窗模块 Kernel_size核心大小      = 5
滑窗模块 Data_vertical图像纵向

# 池化层参数输入

In [19]:
mode=MODE_POOL_Mean
Data_horizontal=24
Data_vertical=Data_horizontal
Kernel_Size=2
# input_picture_buffer 初始化
input_pool_data_size = Data_horizontal*Data_vertical
input_pool_picture_buffer = allocate(shape=(input_pool_data_size,), dtype=np.int8)
for i in range(input_pool_data_size):
    input_pool_picture_buffer[i] = i%Data_vertical+1 #matrix_conv[1][i]
    
for j in range(Data_horizontal):
    print("\n")
    for k in range(Data_vertical):
        print(hex(input_pool_picture_buffer[j*Data_horizontal+k]),"  ", end="")



0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0xc   0xd   0xe   0xf   0x10   0x11   0x12   0x13   0x14   0x15   0x16   0x17   0x18   

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x9   0xa   0xb   0x

In [30]:
Kernel_Size=2
mode=MODE_POOL_Max
# mode=MODE_POOL_Mean
for i in range(6):
    start=time.time()
    Pooling_settings(mode,Data_horizontal,Data_vertical,input_pool_picture_buffer,Kernel_Size,Kernel_Num=1)
    Show_result_data(mode)

滑窗模块 Kernel_size核心大小      = 2
滑窗模块 Data_vertical图像纵向长  = 24
滑窗模块 Data_horizontal图像横向长= 24
滑窗模块 Stride步进               = 2
滑窗模块 Refresh刷新              = 0
 卷积核心大小     2
 卷积核心个数     1
-------------time cost is--------------- 0.00369s
 CNN计算模式   :POOL_Max    
结果输出个数--------- 2304
------ 池化算子结果------ 



0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x16   0x18   

0x2   0x4   0x6   0x8   0xa   0xc   0xe   0x10   0x12   0x14   0x1

# 全连接参数输入

In [11]:
# input_weightfc_buffer 初始化  一次传入FULLCON_Output_Size个数据
FULLCON_Input_Size=100
FULLCON_Output_Size=8
FC_whole_weight_data_size = FULLCON_Input_Size*FULLCON_Output_Size
input_fcweight_buffer = allocate(shape=(FC_whole_weight_data_size,), dtype=np.uint8)
FC_weight_buffer = allocate(shape=(FULLCON_Input_Size,), dtype=np.int64)

for i in range(FC_whole_weight_data_size):
    input_fcweight_buffer[i] = i%FULLCON_Output_Size+1

for j in range(FULLCON_Input_Size):
    for k in range(FULLCON_Output_Size ):
        if(input_fcweight_buffer[j*FULLCON_Output_Size+k]!=0):
            FC_weight_buffer[j]=FC_weight_buffer[j]*256+input_fcweight_buffer[j*FULLCON_Output_Size+k]
            print(hex(input_fcweight_buffer[j*FULLCON_Output_Size+k]),"  ", end="")
    print(hex(FC_weight_buffer[j]),"  ", end="")
    print("-------------------", j,  "\n")
    
FC_whole_act_data_size = FULLCON_Input_Size
FC_act_buffer = allocate(shape=(FC_whole_act_data_size,), dtype=np.int64)

for i in range(FC_whole_act_data_size):
    FC_act_buffer[i] = 1

for j in range(FC_whole_act_data_size):
    print(hex(FC_act_buffer[j]),"  ", end="")
    print("-------------------", j,  "\n")

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 0 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 1 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 2 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 3 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 4 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 5 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 6 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 7 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 8 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------------- 9 

0x1   0x2   0x3   0x4   0x5   0x6   0x7   0x8   0x102030405060708   ------------

In [12]:
mode=MODE_FULLCON
for i in range(1):
    start=time.time()
    Fullconnection_settings(mode,FULLCON_Input_Size,FULLCON_Output_Size,FC_weight_buffer,FC_act_buffer)
    Show_result_data(mode)

-------------time cost is--------------- 0.00152s
 CNN计算模式   :FULLCON  
------- 全连接算子结果---- 

0x320   0x2bc   0x258   0x1f4   0x190   0x12c   0xc8   0x64   

